# Beginners Guide to Text Generation using LSTMs

Text Generation is a type of Language Modelling problem. Language Modelling is the core problem for a number of of natural language processing tasks such as speech to text, conversational system, and text summarization. A trained language model learns the likelihood of occurrence of a word based on the previous sequence of words used in the text. Language models can be operated at character level, n-gram level, sentence level or even paragraph level. In this notebook, I will explain how to create a language model for generating natural language text by implement and training state-of-the-art Recurrent Neural Network. 

### Generating News headlines 

In this kernel, I will be using the dataset of [New York Times Comments and Headlines](https://www.kaggle.com/aashita/nyt-comments) to train a text generation language model which can be used to generate News Headlines


## 1. Import the libraries

As the first step, we need to import the required libraries:

In [3]:
# keras module for building LSTM 
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils as ku 

# set seeds for reproducability
from tensorflow import set_random_seed
from numpy.random import seed
set_random_seed(2)
seed(1)

import pandas as pd
import numpy as np
import string, os 

import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

## 2. Load the dataset

Load the dataset of news headlines

In [4]:
curr_dir = '../input/'
all_headlines = []
for filename in os.listdir(curr_dir):
    if 'Articles' in filename:
        article_df = pd.read_csv(curr_dir + filename)
        all_headlines.extend(list(article_df.headline.values))
        break

all_headlines = [h for h in all_headlines if h != "Unknown"]
len(all_headlines)

829

## 3. Dataset preparation

### 3.1 Dataset cleaning 

In dataset preparation step, we will first perform text cleaning of the data which includes removal of punctuations and lower casing all the words. 

In [6]:
def clean_text(txt):
    txt = "".join(v for v in txt if v not in string.punctuation).lower()
    txt = txt.encode("utf8").decode("ascii",'ignore')
    return txt 

corpus = [clean_text(x) for x in all_headlines]
corpus[:10]

['nfl vs politics has been battle all season long',
 'voice vice veracity',
 'a standups downward slide',
 'new york today a groundhog has her day',
 'a swimmers communion with the ocean',
 'trail activity',
 'super bowl',
 'trumps mexican shakedown',
 'pences presidential pet',
 'fruit of a poison tree']

### 3.2 Generating Sequence of N-gram Tokens

Language modelling requires a sequence input data, as given a sequence (of words/tokens) the aim is the predict next word/token.  

The next step is Tokenization. Tokenization is a process of extracting tokens (terms / words) from a corpus. Python’s library Keras has inbuilt model for tokenization which can be used to obtain the tokens and their index in the corpus. After this step, every text document in the dataset is converted into sequence of tokens. 


In [7]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
token=Tokenizer()

In [8]:
token.fit_on_texts(corpus)

In [10]:
len(token.word_index)

2287

In [31]:
input_=[]
for sent in corpus:
    token_sent=token.texts_to_sequences([sent])[0]
    
    
    for i in range(1,len(token_sent)):
        input_.append(token_sent[:i+1])
    
    



In [32]:
input_[:10]

[[660, 117],
 [660, 117, 72],
 [660, 117, 72, 73],
 [660, 117, 72, 73, 661],
 [660, 117, 72, 73, 661, 662],
 [660, 117, 72, 73, 661, 662, 63],
 [660, 117, 72, 73, 661, 662, 63, 29],
 [660, 117, 72, 73, 661, 662, 63, 29, 210],
 [211, 663],
 [211, 663, 664]]

In [33]:
# Calculate max size
max([len(i) for i in input_])

17

In [34]:
# Padding (Pre)
from tensorflow.keras.preprocessing.sequence import pad_sequences
padded_in=pad_sequences(input_,maxlen=17,padding='pre')
padded_in[:10]

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0, 660, 117],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0, 660, 117,  72],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
        660, 117,  72,  73],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0, 660,
        117,  72,  73, 661],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0, 660, 117,
         72,  73, 661, 662],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0, 660, 117,  72,
         73, 661, 662,  63],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0, 660, 117,  72,  73,
        661, 662,  63,  29],
       [  0,   0,   0,   0,   0,   0,   0,   0, 660, 117,  72,  73, 661,
        662,  63,  29, 210],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0, 211, 663],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         

In [35]:
# Form X and y
X=padded_in[:,:-1]
y=padded_in[:,-1]

In [36]:
X.shape

(4544, 16)

In [37]:
y.shape

(4544,)

In [39]:
# Transform my output into OHE
from tensorflow.keras.utils import to_categorical
y=to_categorical(y,num_classes=2288)

In [40]:
y.shape

(4544, 2288)

In [41]:
# Import models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,LSTM,Embedding

In [42]:
model=Sequential()
model.add(Embedding(2288,100,input_length=16))
model.add(LSTM(169))
model.add(Dense(2288,activation='softmax'))

In [44]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [45]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 16, 100)           228800    
_________________________________________________________________
lstm (LSTM)                  (None, 169)               182520    
_________________________________________________________________
dense (Dense)                (None, 2288)              388960    
Total params: 800,280
Trainable params: 800,280
Non-trainable params: 0
_________________________________________________________________


In [46]:
model.fit(X,y,epochs=100)

Epoch 1/100
4544/4544 [==============================] - 6s 1ms/step - loss: 7.3408 - acc: 0.0341
Epoch 2/100
4544/4544 [==============================] - 3s 696us/step - loss: 6.7678 - acc: 0.0370
Epoch 3/100
4544/4544 [==============================] - 3s 711us/step - loss: 6.5864 - acc: 0.0425
Epoch 4/100
4544/4544 [==============================] - 3s 704us/step - loss: 6.4266 - acc: 0.0502
Epoch 5/100
4544/4544 [==============================] - 3s 696us/step - loss: 6.2562 - acc: 0.0511
Epoch 6/100
4544/4544 [==============================] - 3s 693us/step - loss: 6.0639 - acc: 0.0546
Epoch 7/100
4544/4544 [==============================] - 3s 725us/step - loss: 5.8535 - acc: 0.0658
Epoch 8/100
4544/4544 [==============================] - 3s 724us/step - loss: 5.6299 - acc: 0.0764
Epoch 9/100
4544/4544 [==============================] - 3s 690us/step - loss: 5.3957 - acc: 0.0838
Epoch 10/100
4544/4544 [==============================] - 3s 705us/step - loss: 5.1575 - acc: 0.0933
E

In [52]:
# My training accuracy is 94% which is good.
#Testing.
text = "india"

for i in range(7):
  # tokenize
  token_text = token.texts_to_sequences([text])[0]
  # padding
  padded_token_text = pad_sequences([token_text], maxlen=16, padding='pre')
  # predict
  pos = np.argmax(model.predict(padded_token_text))

  for word,index in token.word_index.items():
    if index == pos:
      text = text + " " + word
      print(text)

india in
india in flatiron
india in flatiron district
india in flatiron district will
india in flatiron district will close
india in flatiron district will close this
india in flatiron district will close this spring


## Improvement Ideas 

As we can see, the model has produced the output which looks fairly fine. The results can be improved further with following points:
- Adding more data
- Fine Tuning the network architecture
- Fine Tuning the network parameters

Thanks for going through the notebook, please upvote if you liked. 